# Clase Práctica 06

# CNN - Redes Neuronales Convolucionales

# Reconocimiento de dígitos escritos a mano



Una demostración popular del uso de las técnicas de Deep Learning es el reconocimiento de objetos en imagenes. El "Hola mundo" del reconocimiento de objetos consiste en utilizar el conjunto de datos MNIST para el reconocimiento de dígitos a mano.

En esta clase práctica se va a desarrollar un modelo de aprendizaje profundo para lograr un rendimiento casi moderno en el reconocimiento de dígitos manuscritos (MNIST) en Python, utilizando la biblioteca de aprendizaje profundo Keras. 

MNIST es una base de datos desarrollada por Yann LeCun, Corinna Cortes y Christopher Burges para evaluar modelos de aprendizaje automático sobre el problema de clasificación de dígitos manuscritos.

La base de datos se construyó a partir de una serie de documentos escaneados disponibles del Instituto Nacional de Estándares y Tecnología (NIST). Aquí es de donde viene el nombre, base de datos NIST Modied o MNIST.

Se tomaron imágenes de dígitos de una gran variedad de documentos escaneados, normalizados en tamaño y centrados. Esto lo convierte en una excelente base de datos para evaluar modelos, lo que le permite al desarrollador enfocarse en el aprendizaje automático con muy poca limpieza de datos o preparación requerida. Cada imagen es un cuadrado de 28x28 píxeles (784 píxeles en total). Se utiliza una división estándar del conjunto de datos para evaluar y comparar modelos, se usan 60,000 imágenes para entrenar un modelo y un conjunto separado de 10,000 imágenes para probarla.

Es una tarea de reconocimiento de dígitos. Como tal, hay 10 dígitos (0 a 9) o 10 clases para predecir. Se puede lograr un error de predicción en el estado del arte de aproximadamente 0.2% con redes neuronales convolucionales profundas.

En esta clase práctica, se realizará la clasificación de la base de datos MNIST usando redes neuronales clásicas y redes neuronales profundas. 

# Cargando la base de datos MNIST en Keras

Keras proporciona un método conveniente para cargar el conjunto de datos MNIST. El conjunto de datos se descarga automáticamente la primera vez que se llama a esta función y se almacena en su directorio de inicio en ~/.keras/datasets/mnist.pkl.gz como un archivo de 15 megabytes. Para demostrar lo fácil que es cargar el conjunto de datos MNIST, primero escribiremos un pequeño script para descargar y visualizar las primeras 4 imágenes en el conjunto de datos de entrenamiento.


In [ ]:
#librerías

from keras.datasets import mnist    # MNIST database
import matplotlib.pyplot as plt

# seleccionar entradas y salidas (etiquetas) de la base MNIST
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# dibujar 4 imágenes
plt.subplot(221)
plt.imshow(X_train[0], cmap=plt.get_cmap('gray'))
plt.subplot(222)
plt.imshow(X_train[1], cmap=plt.get_cmap('gray'))
plt.subplot(223)
plt.imshow(X_train[2], cmap=plt.get_cmap('gray'))
plt.subplot(224)
plt.imshow(X_train[3], cmap=plt.get_cmap('gray'))
# mostrar
plt.show()
print("X_train forma original", X_train.shape)
print("y_train forma original", y_train.shape)

Using TensorFlow backend.


Los datos también pueden ser visualizados de la siguiente manera:

In [ ]:
plt.rcParams['figure.figsize'] = (7,7) # agrando un poco la figura 
# ciclo para dibujar imágenes con su clase
for i in range(9):
    plt.subplot(3,3,i+1)
    plt.imshow(X_train[i], cmap='gray', interpolation='none')
    plt.title("Class {}".format(y_train[i]))

Pensemos lo siguiente:

¿Realmente necesitamos un modelo complejo como una red neuronal convolucional para obtener los mejores resultados con MNIST? ¿Es posible obtener buenos resultados utilizando un modelo de red neuronal muy simple con una sola capa oculta? 

A continuación crearemos un modelo de Perceptrón multicapa simple que logre una tasa de error pequeña. Usaremos esto como base para la comparación con modelos de redes neuronales convolucionales más complejas. Comencemos importando las clases y funciones que necesitaremos.

In [ ]:
import numpy
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import np_utils

seed = 5
numpy.random.seed(seed)

Ahora cargamos la base de datos MNIST. El conjunto de datos de entrenamiento está estructurado como una matriz tridimensional de ancho de imagen, altura de imagen y produndidad. Para un modelo de perceptron multicapa, debemos reducir las imágenes a un vector de píxeles. En este caso, las imágenes de tamaño 28x28 serán vectores de entrada de 784 píxeles (28*28). Podemos hacer esta transformación fácilmente usando la función reshape () de la matriz NumPy. Los valores de píxel son enteros, por lo que los convertimos en valores de punto flotante para que podamos normalizarlos fácilmente en el próximo paso.

Los valores de los píxeles están en escala de grises (entre 0 y 255). Siempre es una buena idea realizar un escalamiento (normalización) de valores de entrada cuando se usan modelos de redes neuronales. Debido a que la escala es bien conocida y se comporta bien, podemos normalizar muy rápidamente los valores de píxeles en el rango 0 y 1 dividiendo cada valor por el máximo de 255.

Finalmente, la variable de salida es un número entero de 0 a 9. Este es un problema de clasificación multiclase ya que deben estar codificadas.  Es una buena práctica usar una codificación "one hot encoding", transformando el vector de enteros de clase en una matriz binaria. Podemos hacerlo fácilmente usando la función auxiliar np_utils.to_categorical() incorporada en Keras.

0 -> [1, 0, 0, 0, 0, 0, 0, 0, 0]

1 -> [0, 1, 0, 0, 0, 0, 0, 0, 0]

2 -> [0, 0, 1, 0, 0, 0, 0, 0, 0]

etc.

In [ ]:
# datos
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# aplanar las imagenes de 28*28 a un vector de 784 
num_pixels = X_train.shape[1] * X_train.shape[2]
X_train = X_train.reshape(X_train.shape[0], num_pixels).astype('float32')
X_test = X_test.reshape(X_test.shape[0], num_pixels).astype('float32')

# normalizar de 0-255 a 0-1
X_train = X_train / 255
X_test = X_test / 255

print("Forma de los datos de Training", X_train.shape)
print("Forma de los datos de Testing ", X_test.shape)

YY_train=y_train # respaldar variables originales sin hot encoding
YY_test=y_test
# one hot 
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

# Definición y ajuste del del modelo de red neuronal simple

Ahora estamos listos para crear nuestro modelo de red neuronal simple. Vamos a definir nuestro modelo en una función. Esto es útil si desea extender el ejemplo más adelante e intentar obtener un mejor desempeño.

El modelo es una red neuronal simple con una capa oculta con el mismo número de neuronas que las entradas (784). Se utiliza una función de activación de rectificador ReLU para las neuronas en el capa oculta. Se utiliza una función de activación de softmax en la capa de salida para convertir las salidas en valores de probabilidad y permitir que se seleccione una clase de las 10, como la predicción de salida del modelo. La pérdida logarítmica se usa como la función de pérdida (crossentropy categórica). Para ajustar los pesos se utiliza el algoritmo de descenso de gradiente ADAM. A continuación se proporciona un resumen de la estructura de la red:

<img src="./images_tutoriales/im1.png">

Ahora podemos ajustar y evaluar el modelo. El modelo se ajusta a 10 épocas con actualizaciones cada 200 imágenes (batch). Los datos de test se utilizan como el conjunto de datos de validación, lo que le permite ver la habilidad del modelo a medida que se entrena. Se utiliza un verbose=2 para reducir la salida a una línea para cada época de entrenamiento. Finalmente, el conjunto de datos de test se utiliza para evaluar el modelo y se imprime una tasa de error de clasificación.

In [ ]:
# modelo_base
def modelo_base():
    # modelo
    model = Sequential()
    model.add(Dense(num_pixels, input_dim=num_pixels, init='normal', activation='relu'))
    model.add(Dense(num_classes, init='normal', activation='softmax'))
    # Compilar
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# generar modelo
model = modelo_base()

# ajustar
model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=10, batch_size=200, verbose=2)

# evaluacion final 
scores = model.evaluate(X_test, y_test, verbose=0)
print(" Error del modelo: %.2f%%" % (100-scores[1]*100))

# Inspeccionando la salida del sistema

Siempre es una buena idea inspeccionar la salida y asegurarse de que todo se vea bien. Aquí veremos algunos ejemplos donde nuestro clasificador lo hace bien, y algunos ejemplos en los que se equivoca. Es importante destacar que se utilizan las variables originales (sin one hot encoding). 

In [ ]:
import numpy as np
# predicción del modelo a los datos de test
predicted_classes = model.predict_classes(X_test)

# verificamos cuales son correctos y cuales no, usando las etiquetas y predicciones 
correct_indices = np.nonzero(predicted_classes == YY_test)[0]
incorrect_indices = np.nonzero(predicted_classes != YY_test)[0]

In [ ]:
# dibujemos algunas predicciones correctas e incorrectas
plt.figure()
for i, correct in enumerate(correct_indices[:9]):
    plt.subplot(3,3,i+1)
    plt.imshow(X_test[correct].reshape(28,28), cmap='gray', interpolation='none')
    plt.title("Predicted {}, Class {}".format(predicted_classes[correct], YY_test[correct]))
    
plt.figure()
for i, incorrect in enumerate(incorrect_indices[:9]):
    plt.subplot(3,3,i+1)
    plt.imshow(X_test[incorrect].reshape(28,28), cmap='gray', interpolation='none')
    plt.title("Predicted {}, Class {}".format(predicted_classes[incorrect], YY_test[incorrect]))

# Parte 2. Redes Neuronales Convolucionales

# Modelo de una CNN simple para MNIST

Ahora que hemos visto cómo cargar el conjunto de datos MNIST y entrenar en él un modelo de Perceptrón multicapa simple, es hora de desarrollar una red neuronal convolucional o CNN, más sofisticada. Keras proporciona una gran capacidad para crear redes neuronales convolucionales. En esta sección, crearemos una CNN simple para MNIST que demuestra cómo utilizar todos los aspectos de una implementación moderna de CNN, incluidas las capas convolucionales, las capas de pooling y las capas de exclusión (dropout). El primer paso es importar las clases y funciones necesarias.

In [ ]:
# CNN
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import matplotlib.pyplot as plt


In [ ]:
# parametros del experimento 
batch_size = 200
num_classes = 10
epochs = 10
# input image dimensions
img_rows, img_cols = 28, 28

A continuación, debemos cargar el conjunto de datos MNIST y remodelarlo para que sea adecuado para el entrenamiento de una CNN. En Keras, las capas utilizadas para las convoluciones bidimensionales esperan valores de píxel con las dimensiones [ancho] [altura][canales] . En el caso de RGB, los canales son 3 para los componentes rojo, verde y azul (3 entradas de imagen, una para cada color). En el caso de MNIST donde los valores de los canales están en escala de grises, la dimensión del píxel se establece en 1.

Como antes, es una buena idea normalizar los valores de píxel en el rango 0 y 1 y hacer one hot para la variable de salida.

In [ ]:
# load data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# cambiar a [samples][fil][col][profundidad]
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

# normalización

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# one hot
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Definición del modelo CNN 

A continuación vamos a definir nuestro modelo de red neuronal. Las redes neuronales convolucionales son más complejas que los perceptrones multicapa estándar, por lo que comenzaremos utilizando una estructura simple para utilizar todos los elementos del estado del arte y obtener resultados de vanguardia. A continuación se resume la arquitectura de la red.

1. La primera capa oculta es una capa convolucional llamada Convolution2D. La capa tiene 32 mapas de características (32 filtros de convolución), de tamaño de 5x5 y una función de activación de rectificación (ReLU). Esta capa recibe las imágenes de entrada. 

2. A continuación, definimos una capa de agrupación (pooling) que toma el valor máximo denominado MaxPooling2D. Se configura con un tamaño de pooling de 2x2.

3. La siguiente capa es una capa de regularización que utiliza el abandono de neuronas denominado dropout. Está configurado para excluir al azar el 20% de las neuronas en la capa para reducir el exceso de información y que el clasificador no memorice todos los ejemplos de entrenamiento.

4. La siguiente es una capa que convierte los datos de la matriz 2D en un vector plano (Flatten). Permite que la salida sea procesada por capas estándar completamente conectadas (fully connected).

5. A continuación, se utiliza una capa totalmente conectada (Dense) con 128 neuronas y una función de activación de rectificación ReLU.

6. Finalmente, la capa de salida tiene 10 neuronas para las 10 clases y una función de activación de softmax para generar predicciones de probabilidad para cada clase.

Como antes, el modelo se entrena utilizando la pérdida logarítmica multiclase y el algoritmo de descenso de gradiente ADAM. A continuación se proporciona una descripción de la estructura de la red.

<img src="./images_tutoriales/im2.png">

Si se quiere visualizar la CNN, puede ser de ayuda el siguiente esquema. Notar que el esquema posee diferentes parámetros de filtros, no posee dropout, etc. Es solo para visualizar el proceso de la CNN. 

<img src="./images_tutoriales/im3.png">

In [ ]:
# modelo
def modelo_cnn():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(5, 5), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = modelo_cnn()
    

El modelo puede ser resumido a continuación.

Luego, evaluamos el modelo de la misma manera que antes con el Perceptrón multicapa. La CNN es más de 10 épocas con un tamaño de lote de 200.

In [ ]:
model.summary()

In [ ]:
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test))


acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochss = range(1, len(acc) + 1)
plt.plot(epochss, acc, 'r--o', label='Training acc')
plt.plot(epochss, val_acc, 'b--x', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochss, loss, 'r--o', label='Training loss')
plt.plot(epochss, val_loss, 'b--x', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()


score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print("CNN Error: %.2f%%" % (100-score[1]*100))

# Código completo


In [ ]:
# CNN
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import matplotlib.pyplot as plt

# parametros del experimento 
batch_size = 200
num_classes = 10
epochs = 20
# input image dimensions
img_rows, img_cols = 28, 28

# load data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# cambiar a [samples][fil][col][profundidad]
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

# normalización

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# one hot
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# modelo
def modelo_cnn():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(5, 5), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    # Compilar el modelo
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = modelo_cnn()

history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test))

plt.figure()
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochss = range(1, len(acc) + 1)
plt.plot(epochss, acc, 'r--o', label='Training acc')
plt.plot(epochss, val_acc, 'b--x', label='Validation acc')
plt.title('Accuracy: Entrenamiento y Validación')
plt.legend()
plt.figure()
plt.plot(epochss, loss, 'r--o', label='Training loss')
plt.plot(epochss, val_loss, 'b--x', label='Validation loss')
plt.title('Loss: Entrenamiento y Validación')
plt.legend()
plt.show()


score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print("CNN Error: %.2f%%" % (100-score[1]*100))

# Mejorando la CNN aumentando el número de capas ocultas

Ahora que hemos visto cómo crear una CNN simple, echemos un vistazo a un modelo capaz de obtener resultados cercanos al estado del arte. Importamos las clases y las funciones, luego cargamos y preparamos los datos de la misma manera que en el ejemplo anterior. Esta vez, hemos definido una arquitectura CNN más grande con capas convolucionales adicionales, de agrupación máxima (pooling) y capas totalmente conectadas. La topología de la red se puede resumir de la siguiente manera.

1. Capa convolucional con 30 mapas de características de tamaño 5 x 5.
2. Agrupar (pooling) la capa tomando el máximo de 2 x 2 parches.
3. Capa convolucional con 15 mapas de características de tamaño 3 x 3.
4. Agrupar (pooling) la capa tomando el máximo de 2 x 2 parches.
5. Capa de abandono (dropout) con una probabilidad del 20%.
6. Aplanar la capa (flatten).
7. Capa totalmente conectada con 128 neuronas y activación de rectificador.
8. Capa totalmente conectada con 50 neuronas y activación de rectificador.
9. Output layer.

A continuación se proporciona una descripción de esta estructura de red más grande:

<img src="./images_tutoriales/im4.png">

Al igual que en los dos experimentos anteriores, el modelo es más de 10 épocas con un tamaño de lote de 200.

Al ejecutar el ejemplo se imprime la precisión en los conjuntos de datos de entrenamiento y validación de cada época y una tasa de error de clasificación final. El modelo tarda unos 60 segundos en ejecutarse por época en una CPU moderna. Este modelo ligeramente más grande que el anterior logra la tasa de error de clasificación respetable del 0,8%.


In [ ]:
# CNN
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import matplotlib.pyplot as plt

# parametros del experimento 
batch_size = 200
num_classes = 10
epochs = 20
# input image dimensions
img_rows, img_cols = 28, 28

# load data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# cambiar a [samples][fil][col][profundidad]
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

# normalización

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')


YY_train=y_train # respaldar variables originales sin hot encoding
YY_test=y_test

XX_train=x_train # respaldar variables originales sin hot encoding
XX_test=x_test


# one hot
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# modelo
def modelo_cnn():
    model = Sequential()
    model.add(Conv2D(30, kernel_size=(5, 5),activation='relu',input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(15, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.20))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    # Compilar el modelo
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = modelo_cnn()

history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test))



In [ ]:
plt.figure()
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochss = range(1, len(acc) + 1)
plt.plot(epochss, acc, 'r--o', label='Training acc')
plt.plot(epochss, val_acc, 'b--x', label='Validation acc')
plt.title('Accuracy: Entrenamiento y Validación')
plt.legend()
plt.figure()
plt.plot(epochss, loss, 'r--o', label='Training loss')
plt.plot(epochss, val_loss, 'b--x', label='Validation loss')
plt.title('Loss: Entrenamiento y Validación')
plt.legend()
plt.show()


score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print("CNN Error: %.2f%%" % (100-score[1]*100))

# Revisión del clasificador CNN y puntajes

Observemos ahora el rendimiento de la red. Para esto vamos a tomar los datos de test que la red no ha visto y veremos que predicciones realiza antes entradas aleatoreas. 

In [ ]:
import numpy as np
index = np.random.choice(list(range(len(x_test))), 1)[0]
im = x_test[index]

import matplotlib.pyplot as plt
print('la imagen de test:')
plt.imshow(im[..., 0], cmap='gray')
plt.show()

print('la clase que predice la red es: ', np.argmax(model.predict(np.reshape(im, [1,28,28,1])), -1)[0])

In [ ]:
import numpy as np
# predicciones de nuestra CNN con los datos de test
predicted_classes = model.predict_classes(x_test)

# revisemos cuales prediccioines son correctas e incorrectas
correct_indices = np.nonzero(predicted_classes == YY_test)[0]
incorrect_indices = np.nonzero(predicted_classes != YY_test)[0]

plt.figure()
for i, correct in enumerate(correct_indices[:9]):
    plt.subplot(3,3,i+1)
    plt.imshow(x_test[correct].reshape(28,28), cmap='gray', interpolation='none')
    plt.title("Predicted {}, Class {}".format(predicted_classes[correct], YY_test[correct]))
    
plt.figure()
for i, incorrect in enumerate(incorrect_indices[:9]):
    plt.subplot(3,3,i+1)
    plt.imshow(x_test[incorrect].reshape(28,28), cmap='gray', interpolation='none')
    plt.title("Predicted {}, Class {}".format(predicted_classes[incorrect], YY_test[incorrect]))

# Matriz de confusión y reportes

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score, precision_score, recall_score 

# classification report
print('Reporte de Clasificación:')
print(classification_report( YY_test,predicted_classes))

# confusion matrix 
cm = confusion_matrix(YY_test,predicted_classes)

# mostrar los resultados
print('Matriz de confusión:')
print(cm)

# Print f1, precision, and recall scores
print('Precision:')
print(precision_score(YY_test, predicted_classes , average="macro"))
print('Recall:')
print(recall_score(YY_test, predicted_classes , average="macro"))
print('F1:')
print(f1_score(YY_test, predicted_classes, average="macro"))
